In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
dir_path = '/content/drive/Othercomputers/macbook-air/TPS-Mar2022/code'
os.chdir(dir_path)

In [ ]:
! pip install japanize-matplotlib
! pip install shap

In [ ]:
import glob
import sys,os
import json
import pprint
import time
import re
import datetime
import pickle
import string
import gc
import warnings
import yaml
import os
warnings.filterwarnings("ignore")
sys.path.append(os.pardir)
sys.path.append('../..')
sys.path.append('../../..')

import numpy as np
import pandas as pd
import pandas_profiling as pdp
import matplotlib.pyplot as plt
import japanize_matplotlib # 日本語対応
import seaborn as sns
# pandasのオプション
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', 5000)
pd.options.display.float_format = '{:.3f}'.format
%matplotlib inline
# sns.set_style('whitegrid')
plt.style.use('fivethirtyeight')

from joblib import Parallel, delayed # よりお手軽にサクっと並列処理を実行出来るモジュール
from tqdm import tqdm, tqdm_notebook # プログレスバーを表示できる
from PIL import Image
tqdm.pandas()

# 外部モジュールを自動的にリロードする
%load_ext autoreload
%autoreload 2

CONFIG_FILE = '../configs/config.yaml'

with open(CONFIG_FILE) as file:
    yml = yaml.safe_load(file)
MODEL_DIR_NAME = yml['SETTING']['MODEL_DIR_NAME']
FEATURE_DIR_NAME = yml['SETTING']['FEATURE_DIR_NAME']
RAW_DIR_NAME = yml['SETTING']['RAW_DIR_NAME']

## 生データ確認

In [ ]:
train = pd.read_csv(RAW_DIR_NAME + 'train.csv')
test = pd.read_csv(RAW_DIR_NAME + 'test.csv')

### memo(時系列データの欠損行の補完)


In [ ]:
# 時系列データにおける欠損行の補完
def df_imputation(df, time_col, freq):
  df['merged_feat'] = df['x'].map(lambda x: str(x) + '_') + df['y'].map(lambda x: str(x) + '_') + df['direction']

  df[time_col] = pd.to_datetime(df[time_col])

  unique_time = pd.DataFrame(df[time_col].unique())
  max_time = unique_time.max()[0]
  min_time = unique_time.min()[0]

  # 完全な時系列 * 特徴量のnp.arrayを作成
  absolute_series_arrary = np.meshgrid(np.array(pd.DataFrame(pd.date_range(start=min_time, end=max_time, freq=freq))), df['merged_feat'].unique())

  absolute_series_index = pd.Series(absolute_series_arrary[0].flatten())
  absolute_series_values = pd.Series(absolute_series_arrary[1].flatten()).str.split('_', expand=True).rename(columns = {
    0: 'x',
    1: 'y',
    2: 'direction'
  })

  absolute_series = pd.concat([absolute_series_index, absolute_series_values], axis=1).rename(columns={0: 'time'})
  absolute_series['x'] = absolute_series['x'].map(int)
  absolute_series['y'] = absolute_series['y'].map(int)

  df_imputation = pd.merge(df, absolute_series, how='outer')
  print(df_imputation.shape)
  return df_imputation.drop(['merged_feat'], axis=1)

In [ ]:
# time_col = 'time'
# freq = '20min'

# df = df_imputation(train, time_col, freq)
# df

### memo(基本特徴量の作成　カテゴリカル変数×数値変数)

In [ ]:
time_col = 'time'
df = train.copy()
df[time_col] = pd.to_datetime(df[time_col])

df['date'] = df[time_col].dt.date
df['pm'] = 0
df.loc[df[time_col].dt.hour>=12, 'pm'] = 1

dropped_cols=df.columns

In [ ]:
agg_cols = ['min', 'max', 'mean', 'std']

cols_set = [['date'], ['date', 'pm'], ['date', 'direction', 'x', 'y']]
for cols in cols_set:
  grp_df = df.groupby(cols)['congestion'].agg(agg_cols)
  
  # 過去のデータの統計量を特徴量とする
  for i in [1, 2, 3]:
    grp_df_shift = grp_df.copy()
    col_name = '_'.join(cols)
    grp_df_shift.columns = [f'{col_name}_shift{i}_{c}' for c in grp_df_shift.columns]
    grp_df_shift = grp_df_shift.reset_index()
    grp_df_shift['date'] = grp_df_shift['date'] + timedelta(days=i)
    df = df.merge(grp_df_shift, on=cols, how='left')

## datasets確認

In [ ]:
def load_datasets_train(feats):
    dfs = [pd.read_pickle(FEATURE_DIR_NAME + f'{f}_train.pkl') for f in feats]
    X_train = pd.concat(dfs, axis=1)
    return X_train

def load_train_y(target):
    df = pd.read_pickle(FEATURE_DIR_NAME + f'{target}_train.pkl')
    return pd.Series(df[target])

def load_datasets_both(feats):
    dfs = [pd.read_pickle(FEATURE_DIR_NAME + f'{f}_train.pkl') for f in feats]
    X_train = pd.concat(dfs, axis=1)
    dfs = [pd.read_pickle(FEATURE_DIR_NAME + f'{f}_test.pkl') for f in feats]
    X_test = pd.concat(dfs, axis=1)
    return X_train, X_test

# 欠損値の確認
def missing_values_table(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [ ]:
features = [
      "diff_3days",
      "datetime_element",
      "accum_minutes",
      "x_y_direction_dummies",
      "agg_shift_by_date"
    ]

target = 'congestion'

In [ ]:
train_x, test_x = load_datasets_both(features)
train_y = load_train_y(target)

In [ ]:
display(train_x, test_x)

## gcs

In [ ]:
import os
dir_path = '/content/drive/Othercomputers/macbook-air/TPS-Mar2022/code'
os.chdir(dir_path)
sys.path.append('./')

In [ ]:
from google.cloud import storage
from gcs_client import StorageClient

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../gcs-key.json'
BUCKET_NAME = 'kaggleops-bucket-msm'
BLOB_NAME = 'data'
directry_path = f'../data/features/'

client = storage.Client()
bucket = client.get_bucket(BUCKET_NAME)
StorageClient.upload_gcs_from_directory(bucket, directry_path, BLOB_NAME)

## モデルデータ確認

In [ ]:
pd.read_pickle(MODEL_DIR_NAME + 'lgb_0306_1211/lgb_0306_1211-pred.pkl')

In [ ]:
# display(train.shape, test.shape)

In [ ]:
plt.bar([n for n in range(1, len(pca.explained_variance_ratio_)+1)], pca.explained_variance_ratio_)

In [ ]:
# display(train.head(), train.tail(), train.shape)
# len(train[train['pca4'] > 0.01])
train.max()

In [ ]:
display(train.describe(), test.describe())

In [ ]:
# 各データの欠損値を確認
display(
    missing_values_table(train),
    missing_values_table(test)
)

## create submission

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
! python ../code/20_run.py